# BBC2-VQE

Most common natural orbital functionals (NOFs) can be represented using the energy expression
\begin{equation}
  E = 2 \sum_p n_p H_{pp} + \sum_{pq} A_{pq} J_{qp} - \sum_{pq} B_{pq} K_{qp} - \sum_{pq} C_{pq} L_{qp}
\end{equation}
with $p$ and $q$ corresponding to spatial natural orbitals, and $n_p \in [0,1]$ denoting the occupation number of the $p^{th}$-orbital. Complementary, the associated hole is given by $h_p = 1 - n_p$. These NOFs can be classified according to the integrals that appear in their functional expression.

**BBC2** is a $JK$-type functional with the following coefficients:

\begin{equation}
    A_{pq} = 2n_p n_q
\end{equation}

\begin{equation}
    B_{pq} = \begin{cases}
                n_p n_q, & \text{if } p \leq F, q \leq F, p \neq q \\
        -\sqrt{n_p n_q}, & \text{if } p > F, q> F, p \neq q \\
         \sqrt{n_p n_q}, & \text{otherwise} \\
  \end{cases}
\end{equation}

\begin{equation}
    C_{pq} = 0
\end{equation}

We start by importing some **libraries**

In [1]:
import pennylane as qml
from pennylane import numpy as pnp

from pennylane import FermiC, FermiA
from pennylane import jordan_wigner

import jax
from jax import numpy as jnp

jax.config.update("jax_enable_x64", True)

For exemplification purposes, we define a **system**, in this case $H_2$ at the bonding distance. We also compute monoelectronic and bielectronic integrals and other useful data.

In [2]:
symbols = ["H", "H"]
geometry = pnp.array([[0.0, 0.0, 0.0], [0.7414, 0.0, 0.0]], requires_grad=False)

mol = qml.qchem.Molecule(symbols, geometry, unit="angstrom")
core, h_MO, I_MO = qml.qchem.electron_integrals(mol)()
E_nuc = core[0]

norb = pnp.shape(h_MO)[0]
qubits = 2 * norb

electrons = mol.n_electrons

# Fermi level
F = int(electrons / 2)

The **ansatz** is built using Hartree-Fock as the reference state and a double excitation gate. Other ansatz can be used for more complex systems.

In [3]:
hf_state = [1] * electrons + [0] * (qubits - electrons)

def ansatz(params):
    qml.BasisState(hf_state, wires=range(4))
    qml.DoubleExcitation(params, wires=[0, 1, 2, 3])

We build the second quantization **operators for the 1RDM** and **map** them to Pauli operators using Jordan-Wigner. Then we create a **circuit to measure the 1RDM**.

Note that we are actually measuring the triangular part of $\Gamma^\alpha$ and ensuring to be in the domain of reals:
\begin{equation}
\hat{\Gamma}^\alpha_{qp} = \frac{\hat{a}_{2q}^\dagger \hat{a}_{2p} + \hat{a}_{2p}^\dagger \hat{a}_{2q}}{2}   
\end{equation}

In [4]:
rdm1_ops = []
for p in range(0, norb):
    for q in range(p, norb):
        cpaq = jordan_wigner(0.5 * (FermiC(2 * p) * FermiA(2 * q) + FermiC(2 * q) * FermiA(2 * p))).simplify()
        #### everything is real by construction
        #### cast coefficients to jax reals to avoid warnings with zero values in imaginary parts
        coeffs = jnp.real(jnp.array(cpaq.terms()[0]))
        obs = cpaq.terms()[1]
        cpaq = coeffs[0] * obs[0]
        for coeff, op in zip(coeffs[1:], obs[1:]):
            cpaq += coeff * op
        ####
        rdm1_ops.append(cpaq)

dev = qml.device("lightning.qubit", wires=qubits)

@qml.qnode(dev)
def rdm1_circuit(params):
    ansatz(params)
    return [qml.expval(op) for op in rdm1_ops]

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


The 1RDM is diagonalized to get the **natural orbital representation**

In [5]:
def get_no_on(rdm1):

    rdm1_aa = jnp.zeros((norb, norb))

    i = -1
    for p in range(0, norb):
        for q in range(p, norb):
            i = i + 1
            rdm1_aa = rdm1_aa.at[p, q].set(rdm1[i])
            rdm1_aa = rdm1_aa.at[q, p].set(rdm1[i])

    n, vecs = jnp.linalg.eigh(rdm1_aa)

    n = n[::-1]
    vecs = vecs[:, ::-1]

    return n, vecs

## BBC2

Here, we implement a function that:
  - Measures (or receives) a 1RDM
  - Diagonalizes the 1RDM to generate occupation numbers and natural orbitals
  - Transforms the integrals and computes $E_\text{NOF}$

In [6]:
def E_BBC2(params, rdm1=None):

    if rdm1 is None:
        rdm1 = rdm1_circuit(params)
    n, vecs = get_no_on(rdm1)

    h_NO = jnp.einsum("ij,ip,jq->pq", h_MO, vecs, vecs, optimize=True)
    J_NO = jnp.einsum("ijkl,ip,jq,kq,lp->pq", I_MO, vecs, vecs, vecs, vecs, optimize=True)
    K_NO = jnp.einsum("ijkl,ip,jp,kq,lq->pq", I_MO, vecs, vecs, vecs, vecs, optimize=True)

    B = jnp.zeros((norb, norb))
    for p in range(F, norb):
        for q in range(F, norb):
            B = B.at[p, q].set(-jnp.sqrt(jnp.abs(n[q] * n[p])))
    for p in range(F):
        for q in range(F):
            B = B.at[p, q].set(n[q] * n[p])
    for p in range(F):
        for q in range(F, norb):
            B = B.at[p, q].set(jnp.sqrt(jnp.abs(n[q] * n[p])))
            B = B.at[q, p].set(jnp.sqrt(jnp.abs(n[p] * n[q])))
    for p in range(norb):
        B = B.at[p, p].set(n[p])

    E1 = 0
    for p in range(norb):
        E1 += 2 * n[p] * h_NO[p, p]

    E2 = 0
    for p in range(norb):
        for q in range(norb):
            E2 += 2 * n[q] * n[p] * J_NO[p, q] - B[q, p] * K_NO[p, q]

    return E_nuc + E1 + E2

We assign a **value to the parameter**. For $H_2$, the value corresponding to the *exact* wavefunction is $\theta=0.22501$.

In [7]:
params = 0.22501

Finally, we evaluate the **energy** $E_\text{BBC2}$

In [8]:
E_BBC2(params)

Array(-1.13838839, dtype=float64)

The **gradient** of ${d E_\text{BBC2}}/{d\theta}$ at this value is

In [9]:
jax.grad(E_BBC2)(params)

Array(-0.01068611, dtype=float64)